# Các thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np

# Các thông số cần thiết

In [3]:
def get_data_summary(df=pd.DataFrame):
    """
    Tính toán các chỉ số thống kê mô tả cho các cột số trong DataFrame.

    Args:
        df (DataFrame): DataFrame chứa dữ liệu.

    Returns:
        summary_stats (DataFrame): DataFrame chứa các chỉ số thống kê cho từng cột số.
    """

    # Cột dữ liệu kiểu số
    numerical_columns = df.select_dtypes(include=np.number).copy()

    # Tính toán các giá trị cần thiết
    summary_stats = numerical_columns.round(1).agg([
        'count',  # Số lượng giá trị không thiếu
        lambda column: ((column.isnull().sum() / column.shape[0]) * 100),  # Tỉ lệ giá trị thiếu
        'min',
        'max',
        'mean',  # Giá trị trung bình
        'median', # Trung vị
        'std',  # Độ lệch chuẩn
        'skew', # Độ lệch
        lambda column: column.quantile(0.25),  # Tứ phân vị thứ nhất
        lambda column: column.quantile(0.75)  # Tứ phân vị thứ ba
    ])

    # Đặt lại tên các hàng để dễ nhận diện
    summary_stats.index = ['Count', 'Missing ratio', 'Min', 'Max', 'Mean', 'Median', 'Std', 'Skewness', 'Lower quartile', 'Upper quartile']

    return summary_stats

# Kiểm tra dữ liệu missing

In [4]:
def handle_missing_data(df=pd.DataFrame, handle=0):
    """
    Tính toán tỉ lệ giá trị thiếu trong DataFrame và xử lý chúng.

    Args:
        df (DataFrame): DataFrame chứa dữ liệu.
        handle (str, optional): Phương thức xử lý giá trị thiếu.
            - 0: Xóa các dòng có ít nhất một giá trị thiếu.
            - 1: Điền vào các giá trị thiếu bằng 0.
            - 2: Điền vào các giá trị thiếu bằng trung bình của cột.
            - 3: Điền bằng phương pháp nội suy tuyến tính.

    Returns:
        tuple: Một tuple chứa hai DataFrame và một Series:
            - new_df (DataFrame): DataFrame sau khi xử lý giá trị thiếu.
            - missing_ratio (Series): Series chứa tỉ lệ giá trị thiếu của các thuộc tính.
            - missing_rows (DataFrame): DataFrame chứa các dòng bị thiếu.
    """
    
    # Xác định tỉ lệ giá trị thiếu
    if df.isna().sum().sum() == 0:
        print('Dữ liệu không có giá trị thiếu.')
    else:
        missing_ratio = df.isnull().mean()

    new_df = df.copy()
    missing_rows = df[df.isnull().any(axis=1)]  # Lấy các dòng có ít nhất một giá trị thiếu

    # Xử lý giá trị thiếu
    if handle == 0:
        new_df = df.dropna()            # Xóa các dòng có ít nhất một giá trị thiếu
    elif handle == 1:
        new_df = df.fillna(0)           # Điền vào các giá trị thiếu bằng 0
    elif handle == 2:
        new_df = df.fillna(df.mean())   # Điền vào các giá trị thiếu bằng trung bình của cột
    elif handle == 3:
        new_df = df.interpolate()       # Điền bằng phương pháp nội suy tuyến tính

    return new_df, missing_ratio, missing_rows

# Kiểm tra dữ liệu duplicates

In [ ]:
def handle_duplicate_data(df=pd.DataFrame, drop=False):
    """
    Xác định các dòng dữ liệu bị trùng lặp trong DataFrame và xóa chúng nếu muốn.

    Args:
        df (DataFrame): DataFrame chứa dữ liệu.
        drop (bool, optional): Nếu True, xóa các dòng bị trùng lặp. Mặc định là False.

    Returns:
        tuple: Một tuple chứa hai DataFrame:
            - new_df (DataFrame): DataFrame sau khi xóa các dòng bị trùng lặp.
            - duplicates (DataFrame): DataFrame chứa các dòng dữ liệu bị trùng lặp.
    """

    # Số dòng dữ liệu bị trùng lặp
    num_dup_rows = df.duplicated().sum()

    new_df = df.copy()
    duplicates = pd.DataFrame()
    if num_dup_rows == 0:
        print('Dữ liệu không bị trùng lặp.')
    else:
        # Các dòng bị trùng lặp
        duplicates = df[df.duplicated()]
        print(f'Dữ liệu có {num_dup_rows} dòng bị trùng lặp.')
        if drop:
            new_df = df.drop_duplicates()
            print('Xóa thành công các dòng bị trùng lặp.')

    return new_df, duplicates

# Kiểm tra dữ liệu outliers

In [6]:
def handle_outlier_data(df=pd.DataFrame, handle=0):
    """
    Xử lý ngoại lai trong DataFrame.

    Args:
        df (DataFrame): DataFrame chứa dữ liệu.
        handle (str): Phương pháp xử lý ngoại lai.
            - 0: Sử dụng phương pháp Z-score.
            - 1: Sử dụng phương pháp tứ phân vị.

    Returns:
        tuple: Một tuple chứa hai DataFrame:
            - new_df (DataFrame): DataFrame sau khi xử lý các giá trị ngoại lai.
            - outliers (DataFrame): DataFrame chứa các giá trị ngoại lai.
    """

    new_df = df.copy()
    numerical_columns = df.select_dtypes(include=np.number).copy()  # Cột dữ liệu kiểu số
    outliers = pd.DataFrame()

    if handle == 0:
        # Tính Z-score
        for column in numerical_columns:
            col_mean = numerical_columns[column].mean()
            col_std = numerical_columns[column].std()
            numerical_columns[column] = (numerical_columns[column] - col_mean) / col_std
        
        # Xác định ngoại lai bằng Z-score
        threshold = 3
        z_scores = numerical_columns.abs() > threshold
        outliers = df[z_scores.any(axis=1)]
        new_df = df[~z_scores.any(axis=1)]
    elif handle == 1:
        # Tính tứ phân vị và xác định ngoại lai
        Q1 = numerical_columns.quantile(0.25)
        Q3 = numerical_columns.quantile(0.75)
        IQR = Q3 - Q1

        # Tính giới hạn trên và dưới
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Tạo DataFrame đánh dấu ngoại lai
        is_outlier = (numerical_columns < lower_bound) | (numerical_columns > upper_bound)
        outliers = df[is_outlier.any(axis=1)]
        new_df = df[~is_outlier.any(axis=1)]
    else:
        print("Phương pháp xử lý ngoại lai không hợp lệ.")

    return new_df, outliers